<img src="../Images/DSC_Logo.png" style="width: 400px;">

# Time Series Theory in Python - Part 2: Stationary Time Series Models

This notebook introduces stationary time series models, including Moving Average (MA), Autoregressive (AR), Autoregressive Moving Average (ARMA), and Autoregressive Integrated Moving Average (ARIMA) models. 

In [ ]:
!pip install statsmodels PythonTsa 

In [ ]:
from PythonTsa.datadir import getdtapath
dtapath=getdtapath()

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning) # Suppress specific warnings

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf
from pandas.plotting import lag_plot
from statsmodels.stats.diagnostic import acorr_ljungbox
import statsmodels.api as sm

## 1. Moving Average (MA) Models

In [ ]:
from statsmodels.tsa.arima_process import arma_generate_sample

Generating MA(2) Model given by the equation:
<span style="font-size: 24px;">$$ X_t = \varepsilon_t + 0.6 \varepsilon_{t-1} - 0.3 \varepsilon_{t-2} $$</span>


With the Python function `arma_generate_sample`, we can get samples from an autoregressive moving average (ARMA) process defined by specified parameters.

In [ ]:
# Define the MA parameters
ma = np.array([1, 0.6, -0.3])  # MA coefficients

# Set the random seed for reproducibility
np.random.seed(123457)

# Generate a sample from the ARMA process
x = arma_generate_sample(ar=[1], ma=ma, nsample=500)  # AR part is set to [1] for no AR component

# Check the type of x (should be a numpy array)
print(type(x))  # Output: <class 'numpy.ndarray'>

# Convert x to a pandas Series
x = pd.Series(x)

# Check the type of x again (now it should be a Series)
print(type(x))  # Output: <class 'pandas.core.series.Series'>

# Plot the time series
x.plot()
plt.title('Generated MA(2) Series')
plt.xlabel('Time')
plt.ylabel('Value')
plt.show()

# Plot ACF and PACF
plot_acf(x, lags=20)
plt.title('ACF of the Generated MA(2) Series')
plt.show()
plot_pacf(x, lags=20)
plt.title('PACF of the Generated MA(2) Series')
plt.show()

# Lag plots for lag 1 and 2
fig = plt.figure()
lag_plot(x, lag=1, ax=fig.add_subplot(211))
plt.title('Lag Plot (Lag 1)')
lag_plot(x, lag=2, ax=fig.add_subplot(212))
plt.title('Lag Plot (Lag 2)')
plt.tight_layout()
plt.show()

# Lag plots for lags 3 and 4
fig = plt.figure()
lag_plot(x, lag=3, ax=fig.add_subplot(211))
plt.title('Lag Plot (Lag 3)')
lag_plot(x, lag=4, ax=fig.add_subplot(212))
plt.title('Lag Plot (Lag 4)')
plt.tight_layout()
plt.show()

## 2. Autoregressive Models

Generating a sample of size (length) 500 from AR(2) Model given by the equation:  
<span style="font-size: 24px;">$$ X_t = 0.8 X_{t-1} - 0.3 X_{t-2} + \varepsilon_t $$</span>  

where  <span style="font-size: 18px;">$ \varepsilon_t \sim \text{iid} \, N(0, 1). $</span>


In [ ]:
# Define the AR parameters (AR(2) model)
ar = np.array([1, -0.8, 0.3])  # AR coefficients

# Set the random seed for reproducibility
np.random.seed(123457)

# Generate a sample from the AR process
x = arma_generate_sample(ar=ar, ma=[1], nsample=500)  # ma=[1] means no MA part in the model

# Convert the generated sample to a pandas Series
x = pd.Series(x)

# Plot the time series
x.plot()
plt.title('Generated AR(2) Sample')
plt.xlabel('Time')
plt.ylabel('Value')
plt.show()

# Plot ACF and PACF
plot_acf(x, lags=20)
plt.title('ACF of the Generated AR(2) Sample')
plt.show()
plot_pacf(x, lags=20)
plt.title('PACF of the Generated AR(2) Sample')
plt.show()

# Lag plot for lag 11
lag_plot(x, lag=11)
plt.title('Lag Plot (Lag 11)')
plt.show()


## 3. Autoregressive Moving Average (ARMA) Models

### **Example 1: Simulating and Building an ARMA(2, 2) Model**

Consider the following ARMA(2,2) model:

<span style="font-size: 18px;">$$
X_t = 0.8 X_{t-1} - 0.6 X_{t-2} + \varepsilon_t + 0.7 \varepsilon_{t-1} + 0.4 \varepsilon_{t-2}
$$</span>

In this model, $ X_t $ is expressed as a linear combination of its previous values and past error terms $( \varepsilon $). We can simulate this ARMA(2,2) model using Python to analyze its properties and behavior.


### Build dataset:

In [ ]:
from statsmodels.tsa.arima.model import ARIMA 

In [ ]:
# Set random seed for reproducibility
np.random.seed(12357)

# Define AR and MA parameters for the ARMA model
ar = np.array([1, -0.8, 0.6])  # AR coefficients
ma = np.array([1, 0.7, 0.4])   # MA coefficients

# Create an ARMA process
arma_process = sm.tsa.ArmaProcess(ar, ma)

# Check for stationarity and invertibility
print("Stationarity:", arma_process.isstationary)  # Check if the process is stationary
print("Invertibility:", arma_process.isinvertible)  # Check if the process is invertible

# Generate a sample from the ARMA process
y = arma_generate_sample(ar=ar, ma=ma, nsample=500)
y = pd.Series(y, name='y')  # Convert the generated data to a pandas Series

# Plot the time series
plt.figure()
y.plot(title='ARMA(2, 2) Time Series', xlabel='Time', ylabel='Value')
plt.grid()
plt.show()

# Plot ACF and PACF
plot_acf(y, lags=20)
plt.show()
plot_pacf(y, lags=20)
plt.show()

# Lag plot for lag 11
lag_plot(y, lag=11)
plt.title('Lag Plot (Lag 11)')
plt.show()

### Choose the best model:

Here we perform a comprehensive search for the best ARMA model through an exploration of multiple combinations of orders `p` and `q`.

Note: ARMA modeling is performed using the Python function ARIMA, where the differencing parameter is set to zero (i.e., order=(p, 0, q)). This fits an ARMA(p, q) model to the data.


In [ ]:
max_p = 6
max_q = 7
model_metrics = []

print("Fitting ARMA models...")  # Debug message

for p in range(max_p + 1):
    for q in range(max_q + 1):
        if p == 0 and q == 0:
            continue  # Skip the case where both p and q are zero
        
        try:
            model = ARIMA(y, order=(p, 0, q)).fit()  # Use ARIMA with d=0 for ARMA
            model_metrics.append((p, q, model.aic, model.bic, model.hqic))  # Store p, q, AIC, BIC, HQIC
            print(f"Fitted ARIMA({p}, 0, {q}) with AIC: {model.aic}, BIC: {model.bic}, HQIC: {model.hqic}")  # Print AIC, BIC, HQIC for each fitted model
        except Exception as e:
            print(f"Error fitting ARIMA({p}, 0, {q}): {e}")

# Convert model metrics to DataFrame for easier analysis
metrics_df = pd.DataFrame(model_metrics, columns=['p', 'q', 'AIC', 'BIC', 'HQIC'])

# Print model metrics
print("\nModel Metrics for Different ARMA Models:")
print(metrics_df)

# Get the best model based on the minimum AIC
if not metrics_df.empty:
    best_model = metrics_df.loc[metrics_df['AIC'].idxmin()]
    print("\nBest ARMA Model (p, q, AIC, BIC, HQIC):", best_model)
else:
    print("No models were fitted successfully.")

In [ ]:
inf=sm.tsa.arma_order_select_ic(y, max_ar=6, max_ma=7, ic=['aic', 'bic', 'hqic'], trend='c') # for statsmodels 0.13.0 and later, trend=’n’

In [ ]:
inf.aic_min_order

In [ ]:
inf.bic_min_order

In [ ]:
inf.hqic_min_order

In [ ]:
arma22=ARIMA(y, order=(2,0,2)).fit()

In [ ]:
print(arma22.summary())

### Fit model:

In [ ]:
# Fit the ARMA(2, 2) model
arma22 = ARIMA(y, order=(2, 0, 2), trend='n').fit()

# Analyze the residuals
resid22 = arma22.resid

# Plot ACF and PACF of the residuals
plot_acf(resid22, lags=20)
plt.show()
plot_pacf(resid22, lags=20)
plt.show()

# Perform the Ljung-Box test for residuals
    # Calculate Ljung-Box test statistics and p-values
ljung_box_results = acorr_ljungbox(resid22, lags=26, return_df=True)
    # Create a plot for the p-values
plt.figure()
plt.plot(ljung_box_results['lb_pvalue'], marker='o', linestyle='-', color='b')
plt.axhline(y=0.05, color='r', linestyle='--')  # 5% significance level
plt.title('Ljung-Box Test P-Values')
plt.xlabel('Lags')
plt.ylabel('P-Value')
plt.xticks(np.arange(0, 31, 1))
plt.xticks(ljung_box_results.index)  # Set x-ticks to all lags
plt.gca().set_xticklabels([str(int(x)) if x % 2 == 0 else '' for x in ljung_box_results.index])
plt.grid()
plt.show()

# Perform the normality test on residuals
from scipy import stats 
normaltest_result = stats.normaltest(resid22)
print("Normality test result:", normaltest_result)

### Predict model:

In [ ]:
# Get predictions for the specified range (start=450, end=509)
pred = arma22.get_prediction(start=450, end=509)
predicts = pred.predicted_mean
predconf = pred.conf_int()

# Combine observed data, predictions, and confidence intervals into a DataFrame
predframe = pd.concat([y[450:], predicts, predconf], axis=1)
predframe.columns = ['Observed', 'Predicted', 'Lower CI', 'Upper CI']

# Plot observed, predicted, and confidence intervals
plt.figure(figsize=(10, 5))
plt.plot(predframe['Observed'], label='Observed', color='blue')
plt.plot(predframe['Predicted'], label='Predicted', color='red')
plt.fill_between(predframe.index, predframe['Lower CI'], predframe['Upper CI'], color='gray', alpha=0.5, label='Confidence Interval')
plt.title('ARMA(2, 2) Predictions and Confidence Intervals')
plt.legend()
plt.show()

### **Example 2: The NAO Index Since January 1950**

The time series dataset "nao" in the folder "Ptsadata" is the monthly mean North Atlantic Oscillation (NAO) index since January 1950. The NAO index is important for weather research and is based on the surface sea level pressure difference between the subtropical (Azores) high and the subpolar low.

In [ ]:
# Load the NAO dataset
nao = pd.read_csv(dtapath + 'nao.csv', header=0)

# Create a time index
timeindex = pd.date_range('1950-01', periods=len(nao), freq='ME')
nao.index = timeindex

# Extract NAO index as a Series
naots = nao['index']  # Ensure 'index' corresponds to the correct column name

# Check the type of the dataframe and series
print(type(nao))    # Should print: <class 'pandas.core.frame.DataFrame'>
print(type(naots))  # Should print: <class 'pandas.core.series.Series'>

# Plot the NAO index time series
naots.plot(title='NAO Index Time Series', xlabel='Date', ylabel='NAO Index')
plt.show()

**Exercise:** Plot ACF and PACF and fit an ARMA(1,0) model to the data.

**Exercise:** Analyze residuals from the ARMA model by receiving the residuals from the fitted model, plotting ACF and PACF, and performing the Ljung-Box test for residuals.

**Exercise:** Use the fitted model for predictions from April 2010 to December 2019. Plot the predictions and confidence intervals.

## 4. Differencing, and Stationarity Test

### KPSS Stationary Test

For application of the KPSS test, in the module `statsmodels.tsa.stattools`, there is a test function `kpss` that is the KPSS stationarity test where the argument `regression='c'` means that the function tests the stationarity of a time series without clear trend and obvious seasonality.

In [ ]:
from statsmodels.tsa.stattools import kpss

### **Example 2 [continued]: The NAO Index Since January 1950**

In [ ]:
# Perform KPSS test for stationarity
kpss_result = kpss(naots, regression="c", nlags=50)
print(kpss_result)

The p-value 0.096 validates that the NAO Index time series has stationarity. Hence, applying ARMA model was appropriate.

### **Example 3: Southern Hemisphere Temperature Volatility Data Series**

The dataset "Southtemperature" in the folder "Ptsadata" is the monthly temperature volatility series (Jan 1850 – Dec 2007) for the southern hemisphere, which were extracted from the database maintained by the University of East Anglia Climatic Research Unit. This dataset is read from left to right. In order to make it belong to the Series class in pandas, we use the function `pd.concat` to connect every row in it. As it has 158 rows, we use a for loop to complete this connection.

In [ ]:
# Load the temperature data using read_csv
tem = pd.read_csv(dtapath + 'Southtemperature.txt', header=None, sep='\s+')

# Concatenate the first two rows
temts = pd.concat([tem.loc[0], tem.loc[1]], ignore_index=True)

# Use a loop to concatenate the remaining rows
for i in range(2, 158):
    temts = pd.concat([temts, tem.loc[i]], ignore_index=True)

# Check the type of temts (should be pandas Series)
print(type(temts))  # Output: <class 'pandas.core.series.Series'>

# Create a date range starting from 1850 with monthly frequency
dates = pd.date_range('1850', periods=len(temts), freq='ME')
temts.index = dates

# Plot the original time series
temts.plot()
plt.title('Southern Hemisphere Temperature Volatility')
plt.xlabel('Year')
plt.ylabel('Temperature Volatility')
plt.show()

# KPSS test for stationarity
kpss_stat, p_value, lags, crit_values = kpss(temts, regression='c', nlags='auto')

# Output the results of the KPSS test
print(f'KPSS Statistic: {kpss_stat}')
print(f'p-value: {p_value}')
print(f'Lags: {lags}')
print('Critical Values:', crit_values)
if p_value < 0.05:
    print("The series is likely non-stationary.")
else:
    print("The series is likely stationary.")

First differencing the time series:

In [ ]:
dt = temts.diff(1).dropna()

# Plot the differenced series
dt.plot()
plt.title('First Difference of Temperature Volatility')
plt.xlabel('Year')
plt.ylabel('Differenced Temperature Volatility')
plt.show()

# Plot ACF using PythonTsa (PACF is not plotted here)
plot_acf(dt, lags=48)
plt.title('ACF of Differenced Temperature Volatility')
plt.show()

# KPSS test for stationarity
kpss_stat, p_value, lags, crit_values = kpss(dt, regression='c', nlags='auto')

# Output the results of the KPSS test
print(f'KPSS Statistic: {kpss_stat}')
print(f'p-value: {p_value}')
print(f'Lags: {lags}')
print('Critical Values:', crit_values)
if p_value < 0.05:
    print("The series is likely non-stationary.")
else:
    print("The series is likely stationary.")

### **Example 4: Chinese Quarterly GDP**

We found that the Chinese Quarterly GDP time series has time series has both trend and seasonality: 

In [ ]:
# Load the data
x = pd.read_csv(dtapath + 'gdpquarterlychina1992.1-2017.4.csv',header=0)
dates = pd.date_range(start='1992',periods=len(x),freq='QE')
x.index=dates

# Plot the time series
x.plot()
plt.title('Chinese Quarterly GDP 1992-2017')
plt.ylabel('billions of RMB')
plt.show()

Since it is the quarterly data, the number of seasons is 4 naturally, therefore seasonally difference it with a lag of 4.

In [ ]:
# Create a date range starting from 1992 with quarterly frequency
dates = pd.date_range(start='1992', periods=len(x), freq='QE')
x.index = dates

# Create a time series from the 'GDP' column
x = pd.Series(x['GDP'])

# Seasonal differencing with lag 4
dx = x.diff(4).dropna()

# Plot the seasonally differenced series
dx.plot(marker='o', ms=3)  # ms refers to marker size
plt.title('Seasonally Differenced GDP (Lag 4)')
plt.xlabel('Date')
plt.ylabel('Differenced GDP')
plt.show()

# First differencing the seasonally differenced series
d1dx = dx.diff(1).dropna()

# Plot the first difference of the seasonally differenced series
d1dx.plot(marker='o', ms=3)
plt.title('First Difference of Seasonally Differenced GDP')
plt.xlabel('Date')
plt.ylabel('Differenced GDP')
plt.show()

# Plot ACF and PACF for the first difference of seasonally differenced series
plot_acf(d1dx, lags=44)
plt.title('ACF of Differenced Series')
plt.show()
plot_pacf(d1dx, lags=44)
plt.title('PACF of Differenced Series')
plt.show()

# KPSS test for stationarity
kpss_stat, p_value, lags, crit_values = kpss(d1dx, regression='c', nlags='auto')

# Output the results of the KPSS test
print(f'KPSS Statistic: {kpss_stat}')
print(f'p-value: {p_value}')
print(f'Lags: {lags}')
print('Critical Values:', crit_values)
if p_value < 0.05:
    print("The series is likely non-stationary.")
else:
    print("The series is likely stationary.")

## 5. Autoregressive Integrated Moving Average (ARIMA) Models

The key difference is that ARMA is for stationary data, while ARIMA can handle non-stationary data through its differencing component.

### **Example 5: Global Annual Mean Surface Air Temperature Changes Series (1880-1985)**

The time series dataset "Global mean surface air temperature changes 1880–1985" (denoted as GMSATC) from the folder `Ptsadata` is from Hansen and Lebedeff (1987) that investigates the global warming issue. 

### Preprocess:

In [ ]:
# Load the dataset
tep = pd.read_csv(dtapath + 'Global mean surface air temp changes 1880-1985.csv', header=None)

# Create a date index
dates = pd.date_range('1880-12', periods=len(tep), freq='A-DEC')
tep.index = dates
tepts = pd.Series(tep[0], name='tep')

# Plot the original time series
plt.plot(tepts, color='b')
plt.title('Global Mean Surface Air Temperature Changes (1880-1985)')
plt.show()

# Differencing the time series
dtepts = tepts.diff(1)
dtepts = dtepts.dropna()
dtepts.name = 'dtep'

# Plot the differenced time series
plt.plot(dtepts, color='b')
plt.title('Differenced Time Series')
plt.show()

# KPSS test for stationarity
kpss_test = kpss(dtepts, regression='c', nlags='auto')
print('KPSS Test Results:', kpss_test)

# Plot ACF and PACF
plot_acf(dtepts, lags=20)
plt.show()
plot_pacf(dtepts, lags=20)
plt.show()

### Choose the best model:

In [ ]:
def choose_model(x, max_p, max_q, ctrl=1.03):
    best_aic = np.inf
    best_order = None
    best_mdl = None

    for p in range(max_p + 1):
        for q in range(max_q + 1):
            try:
                if p == 0 and q == 0:
                    continue
                # Use ARIMA model instead of ARMA
                model = ARIMA(x, order=(p, 0, q))
                results = model.fit()
                aic = results.aic
                if aic < best_aic:
                    best_aic = aic
                    best_order = (p, q)
                    best_mdl = results
            except Exception as e:
                print(f"Model fitting failed for order ({p},{q}) with error: {e}")
                continue

    print(f"Best ARIMA model order: {best_order} with AIC: {best_aic}")
    return best_mdl

# Call the model selection function
best_model = choose_model(dtepts, max_p=7, max_q=7, ctrl=1.03)

# Optionally, display the best model summary
if best_model:
    print(best_model.summary())

### Fit model:

In [ ]:
arma11= ARIMA(dtepts, order=(1,3,0)).fit()
print(arma11.summary())

In [ ]:
# Analyze the residuals
resid11 = arma11.resid
resid11.head(10)

In [ ]:
# Perform the normality test on residuals
normaltest_result = stats.normaltest(resid11)
print("Normality test result:", normaltest_result)

# Perform the Ljung-Box test for residuals
    # Calculate Ljung-Box test statistics and p-values
ljung_box_results = acorr_ljungbox(resid11, lags=20, return_df=True)
    # Create a plot for the p-values
plt.figure()
plt.plot(ljung_box_results['lb_pvalue'], marker='o', linestyle='-', color='b')
plt.axhline(y=0.05, color='r', linestyle='--')  # 5% significance level
plt.title('Ljung-Box Test P-Values')
plt.xlabel('Lags')
plt.ylabel('P-Value')
plt.xticks(ljung_box_results.index)  # Set x-ticks to all lags
plt.gca().set_xticklabels([str(int(x)) if x % 2 == 0 else '' for x in ljung_box_results.index])
plt.grid()
plt.show()

# Q-Q plot
plt.figure()
sm.qqplot(resid11, line='q', ax=plt.gca())
plt.title('Q-Q Plot (Sample vs Theoretical Quantiles)')
plt.grid()

# ACF of Residuals
plt.figure()
plot_acf(resid11, lags=25, ax=plt.gca())  # ACF plot for residuals
plt.title('ACF of Residuals')
plt.grid()

# ACF of Squared Residuals
plt.figure()
plot_acf(np.square(resid11), lags=25, ax=plt.gca())  # ACF plot for squared residuals
plt.title('ACF of Squared Residuals')
plt.grid()

### Predict model:

In [ ]:
# Generate prediction results
pred = arma11.get_prediction(start='1960-12', end='1990-12')

# Extract predicted mean and confidence intervals
predicts = pred.predicted_mean
predconf = pred.conf_int()

# Plot
plt.figure(figsize=(10, 5))
plt.plot(dtepts, label='Observed', color='blue')
predicts.plot(label='Forecast', color='red')
plt.fill_between(predicts.index,
                 predconf.iloc[:, 0],
                 predconf.iloc[:, 1], color='gray', alpha=0.3, label='Confidence Interval')
plt.title('ARIMA Forecast (1960-1990)')
plt.xlabel('Year')
plt.ylabel('Differenced Temperature')
plt.legend()
plt.show()